In [ ]:
"""
Created on Wed Dec 15 13:54 2021

This is an example script to prepare the mask file

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import trange, tqdm
import multimelt.plume_functions as pf
import multimelt.box_functions as bf
import multimelt.create_isf_mask_functions as isfmf

Read in data

In [ ]:
inputpath_metadata = './multimelt/mask_info/'
outputpath_mask = # path where you want to store your mask netcdf file

In [ ]:
file_bed_orig = # xr.DataArray containing the bathymetry (on grid EPSG:3031)
file_draft = # xr.DataArray containing the actual ice draft depth (not smoothed out through a grid cell mean when the ice concentration is <1)
file_msk = # xr.DataArray containing mask: 0 = ocean, 1 = ice shelves, 2 = grounded ice (on grid EPSG:3031)
file_isf_conc = # xr.DataArray containing the ice shelf concentration in each grid cell

xx = file_msk['x']
yy = file_msk['y']

Create the masks for ice shelves/ground/pinning points/grounding line

In [ ]:
whole_ds = isfmf.create_mask_and_metadata_isf(
                    file_msk, # file containing info about the grid (needs to be a domain centered around the South Pole!)                                       
                    -1*file_bed_orig, # negative bathymetry           
                    file_msk, # original mask
                    -1*file_draft, # negative ice draft depth
                    file_isf_conc, # ice shelf concentration
                    False, # not chunked (CAREFUL! chunks not necessarily supported yet)
                    inputpath_metadata+'lonlat_masks.txt', # lon/lat boundaries of the ice shelves
                    outputpath_mask, # output path for output to write out intermediate steps
                    inputpath_metadata + 'iceshelves_metadata_Nico.txt', # file containing name and Rignot data about the different ice shelves
                    inputpath_metadata+'GL_flux_rignot13.csv', # file containing the flux at the grounding line from Rignot et al 2013
                    ground_point ='no', # if 'yes', the grounding line is defined on the ice shelf points at the border to the ground
                    FRIS_one=True, # do you want to count Filchner-Ronne as one ice shelf? True if yes, False if you want to have them as two separate ice shelves
                    write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes', # if you already wrote one of these files, you can set option to 'no'
                    dist=40, # Defines the size of the starting square for the ground mask - should be small if the resolution is coarse and high if the resolution is fine - can be modulated
                    add_fac=120, # Defines additional iterations for the propagation for the ground mask - can be modulated
                    write_metadata = 'yes' # writes out the file with only metadata
                    ) 

# Write to netcdf
print('------- WRITE TO NETCDF -----------')
whole_ds.to_netcdf(outputpath_mask + 'mask_file.nc','w')

Prepare the box characteristics (writes the output directly to files)

In [ ]:
whole_ds = xr.open_dataset(outputpath_mask + 'mask_file.nc')

nonnan_Nisf = whole_ds['Nisf'].where(np.isfinite(whole_ds['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = whole_ds.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True) # only look at ice shelves with area larger than 2500 km2
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
isf_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'dGL', 'dIF', 'latitude', 'longitude', 'isf_name']]
out_2D, out_1D = bf.box_charac_file(file_isf['Nisf'],isf_var_of_int, -1*file_draft, file_isf_conc, outputpath_boxes, max_nb_box=10)

print('------ WRITE TO NETCDF -------')
out_2D.to_netcdf(outputpath_boxes + 'boxes_2D.nc', 'w')
out_1D.to_netcdf(outputpath_boxes + 'boxes_1D.nc', 'w')

Prepare the plume characteristics

In [ ]:
plume_param_options = ['simple','lazero', 'appenB']

plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

# Compute the ice draft
ice_draft_pos = file_draft
ice_draft_neg = -1*ice_draft_pos

plume_charac = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int)

print('------ WRITE TO NETCDF -------')
plume_charac.to_netcdf(outputpath_plumes+'plume_characteristics.nc', 'w')